# (1) Loading Data

Here, we will be loading the data, that we had saved latest in the `Data Extraction.ipynb`

In [199]:
import pandas as pd
import pickle
import numpy as np

In [200]:
df = pickle.load(open('dataset_level2.pkl', 'rb'))

 # (1) Working On The Features Over Which Our Model Will Depend
 
 **Note: Batting_team and Bowling_team we already have so we not going to work upon it.**
 
 These features will be:-
 * batting_team
 * bowling_team
 * city
 * current score
 * ball left
 * wickets left
 * current run rate
 * runs in last five overs
 * final_score (runs_x)

In [201]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,venue
0,2,Sri Lanka,Australia,0.1,0,0,Colombo,R Premadasa Stadium
1,2,Sri Lanka,Australia,0.2,1,0,Colombo,R Premadasa Stadium
2,2,Sri Lanka,Australia,0.3,4,0,Colombo,R Premadasa Stadium
3,2,Sri Lanka,Australia,0.4,1,0,Colombo,R Premadasa Stadium
4,2,Sri Lanka,Australia,0.5,1,0,Colombo,R Premadasa Stadium
...,...,...,...,...,...,...,...,...
120,964,Australia,Sri Lanka,19.3,2,0,NaN,Melbourne Cricket Ground
121,964,Australia,Sri Lanka,19.4,1,0,NaN,Melbourne Cricket Ground
122,964,Australia,Sri Lanka,19.5,1,TD Paine,NaN,Melbourne Cricket Ground
123,964,Australia,Sri Lanka,19.6,2,0,NaN,Melbourne Cricket Ground


## (1.1) Creating a City Column

In [202]:
df.isnull().sum()

match_id               0
batting_team           0
bowling_team           0
ball                   0
runs                   0
player_dismissed       0
city                8671
venue                  0
dtype: int64

In [203]:
df[df['city'].isnull()]['venue'].value_counts()

venue
Dubai International Cricket Stadium        3092
Pallekele International Cricket Stadium    2066
Melbourne Cricket Ground                   1453
Sydney Cricket Ground                       749
Adelaide Oval                               498
Harare Sports Club                          372
Sharjah Cricket Stadium                     249
Sylhet International Cricket Stadium        128
Carrara Oval                                 64
Name: count, dtype: int64

In [204]:
def extractCity(s):
    s = str(s)
    l = s.strip().split()
    return l[0]

In [205]:
cities = np.where(df['city'].isnull(), df['venue'].apply(extractCity), df['city']) 

# Breaking down the where function of numpy library:-
# (1) df['city'].isnull(): this tells the where function to locate all those rows in df where the value of 'city' is null.
# (2) df['venue'].apply(extractCity): if the value of df['city'] at a perticular row is null, then apply 'extractCity' function. Check out it's definition.
# (3) df['city']: this the second argument of the where function, which is definitely required, this tells the where clause to leave those row cell unchanged where the value is not null, if it is null then apply the extractCity function.

In [206]:
df['city'] = cities

In [207]:
df.isnull().sum()

match_id            0
batting_team        0
bowling_team        0
ball                0
runs                0
player_dismissed    0
city                0
venue               0
dtype: int64

In [208]:
df.drop(columns = ['venue'], inplace = True)

In [209]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city
0,2,Sri Lanka,Australia,0.1,0,0,Colombo
1,2,Sri Lanka,Australia,0.2,1,0,Colombo
2,2,Sri Lanka,Australia,0.3,4,0,Colombo
3,2,Sri Lanka,Australia,0.4,1,0,Colombo
4,2,Sri Lanka,Australia,0.5,1,0,Colombo
...,...,...,...,...,...,...,...
120,964,Australia,Sri Lanka,19.3,2,0,Melbourne
121,964,Australia,Sri Lanka,19.4,1,0,Melbourne
122,964,Australia,Sri Lanka,19.5,1,TD Paine,Melbourne
123,964,Australia,Sri Lanka,19.6,2,0,Melbourne


In [210]:
eligible_cities = df['city'].value_counts()[df['city'].value_counts() > 600].index.tolist()

In [211]:
df = df[df['city'].isin(eligible_cities)]

In [212]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city
0,2,Sri Lanka,Australia,0.1,0,0,Colombo
1,2,Sri Lanka,Australia,0.2,1,0,Colombo
2,2,Sri Lanka,Australia,0.3,4,0,Colombo
3,2,Sri Lanka,Australia,0.4,1,0,Colombo
4,2,Sri Lanka,Australia,0.5,1,0,Colombo
...,...,...,...,...,...,...,...
120,964,Australia,Sri Lanka,19.3,2,0,Melbourne
121,964,Australia,Sri Lanka,19.4,1,0,Melbourne
122,964,Australia,Sri Lanka,19.5,1,TD Paine,Melbourne
123,964,Australia,Sri Lanka,19.6,2,0,Melbourne


## (1.2) Creating A Current Score Column

In [213]:
df['current_score'] = df.groupby('match_id')['runs'].cumsum()

C:\Users\wel\AppData\Local\Temp\ipykernel_2304\190681986.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['current_score'] = df.groupby('match_id')['runs'].cumsum()


In [214]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score
0,2,Sri Lanka,Australia,0.1,0,0,Colombo,0
1,2,Sri Lanka,Australia,0.2,1,0,Colombo,1
2,2,Sri Lanka,Australia,0.3,4,0,Colombo,5
3,2,Sri Lanka,Australia,0.4,1,0,Colombo,6
4,2,Sri Lanka,Australia,0.5,1,0,Colombo,7
...,...,...,...,...,...,...,...,...
120,964,Australia,Sri Lanka,19.3,2,0,Melbourne,160
121,964,Australia,Sri Lanka,19.4,1,0,Melbourne,161
122,964,Australia,Sri Lanka,19.5,1,TD Paine,Melbourne,162
123,964,Australia,Sri Lanka,19.6,2,0,Melbourne,164


## (1.3) Creating Over and Balls_left Columns

In [215]:
df['over'] = df['ball'].apply(lambda x : str(x).split('.')[0])
df['ball_no'] = df['ball'].apply(lambda x : str(x).split('.')[1])

C:\Users\wel\AppData\Local\Temp\ipykernel_2304\509197750.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['over'] = df['ball'].apply(lambda x : str(x).split('.')[0])
C:\Users\wel\AppData\Local\Temp\ipykernel_2304\509197750.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ball_no'] = df['ball'].apply(lambda x : str(x).split('.')[1])


In [216]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no
0,2,Sri Lanka,Australia,0.1,0,0,Colombo,0,0,1
1,2,Sri Lanka,Australia,0.2,1,0,Colombo,1,0,2
2,2,Sri Lanka,Australia,0.3,4,0,Colombo,5,0,3
3,2,Sri Lanka,Australia,0.4,1,0,Colombo,6,0,4
4,2,Sri Lanka,Australia,0.5,1,0,Colombo,7,0,5
...,...,...,...,...,...,...,...,...,...,...
120,964,Australia,Sri Lanka,19.3,2,0,Melbourne,160,19,3
121,964,Australia,Sri Lanka,19.4,1,0,Melbourne,161,19,4
122,964,Australia,Sri Lanka,19.5,1,TD Paine,Melbourne,162,19,5
123,964,Australia,Sri Lanka,19.6,2,0,Melbourne,164,19,6


In [217]:
df['balls_bowled'] = (df['over'].astype('int')*6) + df['ball_no'].astype('int')

C:\Users\wel\AppData\Local\Temp\ipykernel_2304\572800509.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['balls_bowled'] = (df['over'].astype('int')*6) + df['ball_no'].astype('int')


In [218]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_bowled
0,2,Sri Lanka,Australia,0.1,0,0,Colombo,0,0,1,1
1,2,Sri Lanka,Australia,0.2,1,0,Colombo,1,0,2,2
2,2,Sri Lanka,Australia,0.3,4,0,Colombo,5,0,3,3
3,2,Sri Lanka,Australia,0.4,1,0,Colombo,6,0,4,4
4,2,Sri Lanka,Australia,0.5,1,0,Colombo,7,0,5,5
...,...,...,...,...,...,...,...,...,...,...,...
120,964,Australia,Sri Lanka,19.3,2,0,Melbourne,160,19,3,117
121,964,Australia,Sri Lanka,19.4,1,0,Melbourne,161,19,4,118
122,964,Australia,Sri Lanka,19.5,1,TD Paine,Melbourne,162,19,5,119
123,964,Australia,Sri Lanka,19.6,2,0,Melbourne,164,19,6,120


In [219]:
df['balls_left'] = 120 - (df['balls_bowled'].astype('int'))

C:\Users\wel\AppData\Local\Temp\ipykernel_2304\1192668968.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['balls_left'] = 120 - (df['balls_bowled'].astype('int'))


In [220]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_bowled,balls_left
0,2,Sri Lanka,Australia,0.1,0,0,Colombo,0,0,1,1,119
1,2,Sri Lanka,Australia,0.2,1,0,Colombo,1,0,2,2,118
2,2,Sri Lanka,Australia,0.3,4,0,Colombo,5,0,3,3,117
3,2,Sri Lanka,Australia,0.4,1,0,Colombo,6,0,4,4,116
4,2,Sri Lanka,Australia,0.5,1,0,Colombo,7,0,5,5,115
...,...,...,...,...,...,...,...,...,...,...,...,...
120,964,Australia,Sri Lanka,19.3,2,0,Melbourne,160,19,3,117,3
121,964,Australia,Sri Lanka,19.4,1,0,Melbourne,161,19,4,118,2
122,964,Australia,Sri Lanka,19.5,1,TD Paine,Melbourne,162,19,5,119,1
123,964,Australia,Sri Lanka,19.6,2,0,Melbourne,164,19,6,120,0


In [221]:
df['balls_left'] = df['balls_left'].apply(lambda x : 0 if x<0 else x)

C:\Users\wel\AppData\Local\Temp\ipykernel_2304\3349110332.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['balls_left'] = df['balls_left'].apply(lambda x : 0 if x<0 else x)


In [222]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_bowled,balls_left
0,2,Sri Lanka,Australia,0.1,0,0,Colombo,0,0,1,1,119
1,2,Sri Lanka,Australia,0.2,1,0,Colombo,1,0,2,2,118
2,2,Sri Lanka,Australia,0.3,4,0,Colombo,5,0,3,3,117
3,2,Sri Lanka,Australia,0.4,1,0,Colombo,6,0,4,4,116
4,2,Sri Lanka,Australia,0.5,1,0,Colombo,7,0,5,5,115
...,...,...,...,...,...,...,...,...,...,...,...,...
120,964,Australia,Sri Lanka,19.3,2,0,Melbourne,160,19,3,117,3
121,964,Australia,Sri Lanka,19.4,1,0,Melbourne,161,19,4,118,2
122,964,Australia,Sri Lanka,19.5,1,TD Paine,Melbourne,162,19,5,119,1
123,964,Australia,Sri Lanka,19.6,2,0,Melbourne,164,19,6,120,0


In [223]:
df.drop(columns = ['over', 'ball_no', 'balls_bowled'], inplace = True)

C:\Users\wel\AppData\Local\Temp\ipykernel_2304\1663745572.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns = ['over', 'ball_no', 'balls_bowled'], inplace = True)


In [224]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,balls_left
0,2,Sri Lanka,Australia,0.1,0,0,Colombo,0,119
1,2,Sri Lanka,Australia,0.2,1,0,Colombo,1,118
2,2,Sri Lanka,Australia,0.3,4,0,Colombo,5,117
3,2,Sri Lanka,Australia,0.4,1,0,Colombo,6,116
4,2,Sri Lanka,Australia,0.5,1,0,Colombo,7,115
...,...,...,...,...,...,...,...,...,...
120,964,Australia,Sri Lanka,19.3,2,0,Melbourne,160,3
121,964,Australia,Sri Lanka,19.4,1,0,Melbourne,161,2
122,964,Australia,Sri Lanka,19.5,1,TD Paine,Melbourne,162,1
123,964,Australia,Sri Lanka,19.6,2,0,Melbourne,164,0


## (1.4) Creating a Wickets_left Column

In [225]:
df['player_dismissed'] = df['player_dismissed'].apply(lambda x: 0 if x == '0' else 1)
df['player_dismissed'] = df['player_dismissed'].astype('int')
df['player_dismissed'] = df.groupby('match_id')['player_dismissed'].cumsum()
df['wickets_left'] = 10 - df['player_dismissed']

C:\Users\wel\AppData\Local\Temp\ipykernel_2304\4155877027.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['player_dismissed'] = df['player_dismissed'].apply(lambda x: 0 if x == '0' else 1)
C:\Users\wel\AppData\Local\Temp\ipykernel_2304\4155877027.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['player_dismissed'] = df['player_dismissed'].astype('int')
C:\Users\wel\AppData\Local\Temp\ipykernel_2304\4155877027.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

In [226]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,balls_left,wickets_left
0,2,Sri Lanka,Australia,0.1,0,0,Colombo,0,119,10
1,2,Sri Lanka,Australia,0.2,1,0,Colombo,1,118,10
2,2,Sri Lanka,Australia,0.3,4,0,Colombo,5,117,10
3,2,Sri Lanka,Australia,0.4,1,0,Colombo,6,116,10
4,2,Sri Lanka,Australia,0.5,1,0,Colombo,7,115,10
...,...,...,...,...,...,...,...,...,...,...
120,964,Australia,Sri Lanka,19.3,2,5,Melbourne,160,3,5
121,964,Australia,Sri Lanka,19.4,1,5,Melbourne,161,2,5
122,964,Australia,Sri Lanka,19.5,1,6,Melbourne,162,1,4
123,964,Australia,Sri Lanka,19.6,2,6,Melbourne,164,0,4


## (1.5) Creating A Current-Run-Rate (Crr) Column

In [227]:
df['crr'] = (df['current_score']*6)/(120 - df['balls_left'])

C:\Users\wel\AppData\Local\Temp\ipykernel_2304\556160272.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['crr'] = (df['current_score']*6)/(120 - df['balls_left'])


In [228]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,balls_left,wickets_left,crr
0,2,Sri Lanka,Australia,0.1,0,0,Colombo,0,119,10,0.000000
1,2,Sri Lanka,Australia,0.2,1,0,Colombo,1,118,10,3.000000
2,2,Sri Lanka,Australia,0.3,4,0,Colombo,5,117,10,10.000000
3,2,Sri Lanka,Australia,0.4,1,0,Colombo,6,116,10,9.000000
4,2,Sri Lanka,Australia,0.5,1,0,Colombo,7,115,10,8.400000
...,...,...,...,...,...,...,...,...,...,...,...
120,964,Australia,Sri Lanka,19.3,2,5,Melbourne,160,3,5,8.205128
121,964,Australia,Sri Lanka,19.4,1,5,Melbourne,161,2,5,8.186441
122,964,Australia,Sri Lanka,19.5,1,6,Melbourne,162,1,4,8.168067
123,964,Australia,Sri Lanka,19.6,2,6,Melbourne,164,0,4,8.200000


## (1.6) Creating A Last_five Column

This would store the runs that are scored in the last_five overs.

> **Steps to do this:**
> * First we will divide the matches into different groups using the `groupby` clause
> * Then wee would access all the unique match ids one-by-by.
> * Having all the unique match ids we can access a perticular match id from the list of `groups` using the `.get_group(<id_number>)`.
> * Then we can use the `.rolling` function that creates the rolling window object. This rolling window object is of definite size *let's say size = Y*, the rolling window wll pick up *Y* rows from the dataframe and you can apply any function over these *Y* rows.
> * Here, what we do is that, we call over the values of the column of `df['rows']` and then apply the `.sum()` function so as find the sum for each value.
> * Finally, we convert the values to a list and add that list to our dataframe *here df*.

**Note:** Before adding any column to the `dataframe` we have to first check that all the values are in a form of `list`.

**Note***: Differnce between `extend()` and `append()` function for a dataframe is that `.append()`  is used to add a new row to a dataframe while `.extend()` function is used to add a new column to a dataframe.

In [229]:
groups = df.groupby('match_id')

In [230]:
groups

In [231]:
match_ids = df['match_id'].unique()

In [232]:
match_ids

array([  2,   3,   5,   6,   8,   9,  11,  12,  15,  38,  39,  40,  41,
        42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,
        55,  57,  58,  59,  60,  72,  73,  74,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  98,  99, 100, 101, 102, 106, 107, 108, 129,
       130, 131, 132, 133, 134, 135, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 150, 152, 155, 156, 157, 158, 159, 160, 161,
       162, 163, 164, 166, 167, 168, 170, 171, 172, 174, 176, 177, 178,
       180, 193, 194, 195, 196, 200, 201, 214, 215, 217, 218, 219, 220,
       224, 225, 226, 227, 228, 232, 236, 245, 246, 247, 250, 251, 252,
       253, 254, 255, 256, 257, 258, 261, 265, 266, 267, 271, 274, 275,
       276, 289, 290, 291, 293, 295, 296, 297, 298, 299, 300, 301, 302,
       303, 304, 305, 306, 307, 308, 309, 310, 311, 313, 314, 315, 317,
       318, 320, 323, 324, 325, 326, 327, 330, 331, 334, 335, 336, 337,
       340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 351, 35

In [247]:
last_five = []
for id in match_ids:
    last_five.extend(groups.get_group(id).rolling(window = 30)['runs'].sum().tolist())

In [248]:
df['last_five'] = last_five

C:\Users\wel\AppData\Local\Temp\ipykernel_2304\1595651733.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['last_five'] = last_five


In [253]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,balls_left,wickets_left,crr,last_five
0,2,Sri Lanka,Australia,0.1,0,0,Colombo,0,119,10,0.000000,NaN
1,2,Sri Lanka,Australia,0.2,1,0,Colombo,1,118,10,3.000000,NaN
2,2,Sri Lanka,Australia,0.3,4,0,Colombo,5,117,10,10.000000,NaN
3,2,Sri Lanka,Australia,0.4,1,0,Colombo,6,116,10,9.000000,NaN
4,2,Sri Lanka,Australia,0.5,1,0,Colombo,7,115,10,8.400000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
120,964,Australia,Sri Lanka,19.3,2,5,Melbourne,160,3,5,8.205128,46.0
121,964,Australia,Sri Lanka,19.4,1,5,Melbourne,161,2,5,8.186441,45.0
122,964,Australia,Sri Lanka,19.5,1,6,Melbourne,162,1,4,8.168067,46.0
123,964,Australia,Sri Lanka,19.6,2,6,Melbourne,164,0,4,8.200000,47.0


## (1.8) Calculating The Final Runs Scored In That Match. 

The `.merge()` function in pandas is used to combine two or more dataframes into a single dataframe based on a common set of columns (or index).

*The syntax for the .merge() function is as follows:*

**merged_df = pd.merge(left_df, right_df, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=True, suffixes=('_x', '_y'), copy=True, indicator=False, validate=None)**

where:-
> `left_df` and `right_df` are the two dataframes to be merged

> `how` specifies the type of merge to be performed. It can be 'inner' (default), 'outer', 'left', or 'right'
on specifies the column(s) to merge on. If not specified, the function will merge on columns with the same name in both dataframes

> `left_on` and `right_on specify` the column(s) in the left and right dataframes, respectively, to merge on if they have different names

> `left_index` and `right_index` specify whether to use the left or right dataframe's index as the merge key

> `sort` specifies whether to sort the merged dataframe by the merge key(s)

> `suffixes` specifies the suffixes to add to overlapping column names in the merged dataframe

> `copy` specifies whether to copy the data into a new dataframe or modify the original dataframes

> `indicator` specifies whether to add a column to the merged dataframe indicating which dataframe each row came from

> `validate` specifies whether to check if the merge keys are unique in both dataframes

Here, `runs_x` denote the total number of runs scored in that match, `runs_y` denote the total number of runs scored in a perticular ball.


In [259]:
df = df.groupby('match_id')['runs'].sum().reset_index().merge(df, on = "match_id")

In [263]:
df = df[['batting_team', 'bowling_team', 'city', 'current_score', 'balls_left', 'wickets_left', 'crr', 'last_five', 'runs_x']]

In [264]:
final_df = df.copy()

In [265]:
final_df

,batting_team,bowling_team,city,current_score,balls_left,wickets_left,crr,last_five,runs_x
0,Sri Lanka,Australia,Colombo,0,119,10,0.000000,NaN,128
1,Sri Lanka,Australia,Colombo,1,118,10,3.000000,NaN,128
2,Sri Lanka,Australia,Colombo,5,117,10,10.000000,NaN,128
3,Sri Lanka,Australia,Colombo,6,116,10,9.000000,NaN,128
4,Sri Lanka,Australia,Colombo,7,115,10,8.400000,NaN,128
...,...,...,...,...,...,...,...,...,...
50604,Australia,Sri Lanka,Melbourne,160,3,5,8.205128,46.0,168
50605,Australia,Sri Lanka,Melbourne,161,2,5,8.186441,45.0,168
50606,Australia,Sri Lanka,Melbourne,162,1,4,8.168067,46.0,168
50607,Australia,Sri Lanka,Melbourne,164,0,4,8.200000,47.0,168


# (2) Removing All The Null Values From The Dataset (Final_df)

In [267]:
final_df.isnull().sum()

batting_team         0
bowling_team         0
city                 0
current_score        0
balls_left           0
wickets_left         0
crr                  0
last_five        12053
runs_x               0
dtype: int64

In [268]:
final_df.dropna(inplace = True)

In [269]:
final_df

,batting_team,bowling_team,city,current_score,balls_left,wickets_left,crr,last_five,runs_x
29,Sri Lanka,Australia,Colombo,39,91,9,8.068966,39.0,128
30,Sri Lanka,Australia,Colombo,43,90,9,8.600000,43.0,128
31,Sri Lanka,Australia,Colombo,43,89,9,8.322581,42.0,128
32,Sri Lanka,Australia,Colombo,44,88,9,8.250000,39.0,128
33,Sri Lanka,Australia,Colombo,44,87,9,8.000000,38.0,128
...,...,...,...,...,...,...,...,...,...
50604,Australia,Sri Lanka,Melbourne,160,3,5,8.205128,46.0,168
50605,Australia,Sri Lanka,Melbourne,161,2,5,8.186441,45.0,168
50606,Australia,Sri Lanka,Melbourne,162,1,4,8.168067,46.0,168
50607,Australia,Sri Lanka,Melbourne,164,0,4,8.200000,47.0,168


In [270]:
final_df.isnull().sum()

batting_team     0
bowling_team     0
city             0
current_score    0
balls_left       0
wickets_left     0
crr              0
last_five        0
runs_x           0
dtype: int64

# (3) Shuffling The Dataframe

In order to prevent our model from developing any biased we shuffle our dataframe, using the `.sample()` function.

*The syntax for the .sample() method is as follows:*

**df.sample(n=None, frac=None, replace=False, weights=None, random_state=None, axis=None)**

> where:

> * `n` specifies the number of rows to sample. If both n and frac are not specified, a single row is randomly selected.
> * `frac` specifies the fraction of rows to sample, with a value between 0 and 1. If both n and frac are not specified, a single row is randomly selected.
> * `replace` specifies whether sampling should be done with replacement. The default is False, meaning each row is selected only once. If True, the same row can be selected multiple times.
> * `weights` specifies the weight of each row. Rows with higher weights are more likely to be selected. The default is None, meaning all rows have equal weights.
> * `random_state` specifies the random seed used to generate the random samples. This ensures that the same sample is generated every time the method is called with the same arguments.
> * `axis` specifies whether to sample rows (axis=0, default) or columns (axis=1).


In [274]:
final_df.shape[0] # this gives you the total number of rows in the dataframe

38556

In [275]:
final_df.shape[1] # this gives you the total number of columns in the dataframe.

9

In [276]:
final_df = final_df.sample(final_df.shape[0])

In [277]:
final_df

,batting_team,bowling_team,city,current_score,balls_left,wickets_left,crr,last_five,runs_x
21721,Pakistan,New Zealand,Christchurch,152,9,5,8.216216,46.0,183
30305,India,Bangladesh,Nottingham,112,34,9,7.813953,40.0,180
10284,Bangladesh,India,Mirpur,110,18,5,6.470588,34.0,138
20552,Australia,India,Sydney,85,53,8,7.611940,30.0,171
9109,India,Sri Lanka,Mirpur,49,67,9,5.547170,34.0,130
...,...,...,...,...,...,...,...,...,...
6839,South Africa,Bangladesh,Mirpur,54,82,10,8.526316,36.0,169
29582,Sri Lanka,Pakistan,London,135,14,5,7.641509,33.0,150
14835,Pakistan,South Africa,Centurion,154,34,8,10.744186,63.0,195
18837,New Zealand,Bangladesh,Pallekele,97,48,9,8.083333,52.0,191


In [278]:
final_df.sample(2)

,batting_team,bowling_team,city,current_score,balls_left,wickets_left,crr,last_five,runs_x
16715,India,South Africa,Colombo,61,66,7,6.777778,36.0,152
32260,India,Pakistan,Johannesburg,50,74,8,6.521739,25.0,157


In [280]:
!pip install scikit-learn


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



     ---------------------------------------- 8.3/8.3 MB 2.5 MB/s eta 0:00:00
     ---------------------------------------- 42.2/42.2 MB 1.4 MB/s eta 0:00:00
     -------------------------------------- 298.0/298.0 kB 1.2 MB/s eta 0:00:00


In [282]:
pip install --upgrade pip


     ---------------------------------------- 2.1/2.1 MB 1.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.3.1
    Uninstalling pip-22.3.1:
      Successfully uninstalled pip-22.3.1


In [285]:
X = final_df.drop(columns = ['runs_x'])
y = final_df['runs_x']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

In [287]:
!pip install xgboost

     ---------------------------------------- 0.0/70.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/70.9 MB ? eta -:--:--
     ---------------------------------------- 0.1/70.9 MB 1.2 MB/s eta 0:01:00
     ---------------------------------------- 0.2/70.9 MB 1.3 MB/s eta 0:00:54
     ---------------------------------------- 0.2/70.9 MB 1.3 MB/s eta 0:00:57
     ---------------------------------------- 0.3/70.9 MB 1.2 MB/s eta 0:01:00
     ---------------------------------------- 0.3/70.9 MB 1.2 MB/s eta 0:00:58
     ---------------------------------------- 0.4/70.9 MB 1.3 MB/s eta 0:00:56
     ---------------------------------------- 0.5/70.9 MB 1.2 MB/s eta 0:00:58
     ---------------------------------------- 0.5/70.9 MB 1.3 MB/s eta 0:00:55
     ---------------------------------------- 0.6/70.9 MB 1.3 MB/s eta 0:00:55
     ---------------------------------------- 0.7/70.9 MB 1.3 MB/s eta 0:00:55
     ---------------------------------------- 0.7/70.9 MB 1.3 MB/s

In [288]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_absolute_error


In [289]:
trf = ColumnTransformer([
    ('trf', OneHotEncoder(sparse = False, drop = 'first'), ['batting_team', 'bowling_team', 'city'])
],
    remainder = 'passthrough')

In [292]:
pipe = Pipeline(steps = [
    ('step1', trf),
    ('step2', StandardScaler()),
    ('step3', XGBRegressor(n_estimators = 1000, learning_rate = 0.2, max_depth = 12, random_state = 1))
])

In [293]:
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
print(r2_score(y_test, y_pred))
print(mean_absolute_error(y_test, y_pred))

C:\Users\wel\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


0.9875036680825324
1.680417989299505


In [296]:
pickle.dump(pipe, open('pipe.pkl', 'wb'))

In [299]:
final_df

,batting_team,bowling_team,city,current_score,balls_left,wickets_left,crr,last_five,runs_x
21721,Pakistan,New Zealand,Christchurch,152,9,5,8.216216,46.0,183
30305,India,Bangladesh,Nottingham,112,34,9,7.813953,40.0,180
10284,Bangladesh,India,Mirpur,110,18,5,6.470588,34.0,138
20552,Australia,India,Sydney,85,53,8,7.611940,30.0,171
9109,India,Sri Lanka,Mirpur,49,67,9,5.547170,34.0,130
...,...,...,...,...,...,...,...,...,...
6839,South Africa,Bangladesh,Mirpur,54,82,10,8.526316,36.0,169
29582,Sri Lanka,Pakistan,London,135,14,5,7.641509,33.0,150
14835,Pakistan,South Africa,Centurion,154,34,8,10.744186,63.0,195
18837,New Zealand,Bangladesh,Pallekele,97,48,9,8.083333,52.0,191
